In [ ]:
# Формат вывода ячейки

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Введение в MapReduce модель на Python


In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [ ]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [ ]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [ ]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [ ]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [ ]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [ ]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [ ]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [ ]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [ ]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [ ]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [ ]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [ ]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.905589809636405),
 (1, 2.905589809636405),
 (2, 2.905589809636405),
 (3, 2.905589809636405),
 (4, 2.905589809636405)]

## Inverted index

In [ ]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [ ]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [ ]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [ ]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)]),
 (1, [])]

## TeraSort

In [ ]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.0059671639991895065),
   (None, 0.07724245496172),
   (None, 0.08440804135613444),
   (None, 0.13575647907181598),
   (None, 0.14404826813474803),
   (None, 0.21204275967955666),
   (None, 0.21869633101751806),
   (None, 0.25055756276216923),
   (None, 0.28642389538931257),
   (None, 0.3834487515438496),
   (None, 0.3913614390023946),
   (None, 0.4041378102237341),
   (None, 0.41854626274930695),
   (None, 0.4704310153549396),
   (None, 0.4776995227348928),
   (None, 0.48992216726013693)]),
 (1,
  [(None, 0.5005353544023048),
   (None, 0.5135664686748047),
   (None, 0.53391984417089),
   (None, 0.5587932025401512),
   (None, 0.5673804905854288),
   (None, 0.6926646597910275),
   (None, 0.7237444251339501),
   (None, 0.7557883138083207),
   (None, 0.785709769245918),
   (None, 0.7937098630029404),
   (None, 0.7942646850708935),
   (None, 0.9160468126494941),
   (None, 0.9618068292060864),
   (None, 0.9820764489731459)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [4]:
import random
from functools import reduce
from typing import List

def MAP(numList: List[int]) -> int:
    return max(numList)

def REDUCE(max1: int, max2: int) -> int:
    return max(max1, max2)

def RECORDREADER(count: int) -> List[int]:
    return [random.randint(0, 100) for _ in range(count)]

# Генерация нескольких списков случайных чисел
record = RECORDREADER(10)

print("Record:", record)

# Выполнение Map: нахождение максимального значения в каждом элементе
mapped_values = map(MAP, [record])

# Выполнение Reduce: нахождение максимального значения среди всех элементов
result = reduce(REDUCE, mapped_values)

print("Max value:", result)

Record: [70, 99, 40, 42, 1, 9, 40, 89, 73, 91]
Max value: 99


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [7]:
import random
from functools import reduce
from typing import List, Iterator, Tuple

def MAP(num: int) -> Tuple[int, int]:
    return (1, num)

def REDUCE(key: int, nums: Iterator[Tuple[int, int]]) -> float:
    total_sum = 0
    count = 0
    for _, n in nums:  # Перебираем только значения, пропуская ключи
        total_sum += n
        count += 1
    yield total_sum / count if count else 0

def RECORDREADER(count: int) -> List[int]:
    return [random.randint(0, 100) for _ in range(count)]

# Генерация списка случайных чисел
record = RECORDREADER(10)
print("Record:", record)

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(MAP, record))
print("MAP output:", map_output)

# Применение функции REDUCE к результату группировки
reduce_output = list(REDUCE(1, map_output))  # Группируем по фиктивному ключу 1
print("Reduce output:", reduce_output[0])

Record: [2, 59, 91, 42, 65, 14, 55, 47, 43, 46]
MAP output: [(1, 2), (1, 59), (1, 91), (1, 42), (1, 65), (1, 14), (1, 55), (1, 47), (1, 43), (1, 46)]
Reduce output: 46.4


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [ ]:
import random
from functools import reduce
from typing import List, Iterator, Tuple

def MAP(num: int) -> Tuple[int, int]:
    """
    Функция MAP для создания пары ключ-значение.
    Ключ равен num % 2, а значение — это входное число.
    """
    return (num % 2, num)

def REDUCE(key: int, nums: Iterator[int]) -> float:
    """
    Функция REDUCE для вычисления среднего значения из итератора чисел.
    """
    total_sum = 0
    count = 0
    for n in nums:
        total_sum += n
        count += 1
    return total_sum / count if count else 0

def RECORDREADER(count: int) -> List[int]:
    """
    Функция для генерации случайных чисел в диапазоне от 0 до 100.
    """
    return [random.randint(0, 100) for _ in range(count)]

def group_by_key(iterable: List[Tuple[int, int]]):
    """
    Группирует данные по ключу (первый элемент пары) и возвращает список кортежей.
    """
    grouped_data = {}
    for key, value in iterable:
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

# Генерация списка случайных чисел
record = RECORDREADER(10)
print("Record:", record)

# Применение функции MAP к результату функции RECORDREADER
map_output = list(map(MAP, record))
print("MAP output:", map_output)

# Группировка по ключу и преобразование в список
shuffle_output = group_by_key(map_output)
print("Shuffle output:", shuffle_output)

# Применение функции REDUCE к результату группировки
reduce_output = [REDUCE(key, values) for key, values in shuffle_output]

print("Reduce output:", reduce_output)

Record: [8, 44, 53, 71, 17, 43, 73, 8, 33, 6]
MAP output: [(0, 8), (0, 44), (1, 53), (1, 71), (1, 17), (1, 43), (1, 73), (0, 8), (1, 33), (0, 6)]
Shuffle output: [(0, [8, 44, 8, 6]), (1, [53, 71, 17, 43, 73, 33])]
Reduce output: [16.5, 48.333333333333336]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [ ]:
from typing import Iterator, List, Callable, Tuple, Optional
from collections import defaultdict
import numpy as np

def flatten(nested_iterable: Iterator[Iterator]) -> Iterator:
    """
    Функция для "выравнивания" вложенной структуры данных.
    """
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable: List[Tuple[int, str]]) -> List[Tuple[int, List[str]]]:
    """
    Группировка данных по ключу.
    """
    grouped_data = defaultdict(list)
    for key, value in iterable:
        grouped_data[key].append(value)
    return list(grouped_data.items())

def groupbykey_distributed(map_partitions: Iterator[Iterator[Tuple[int, str]]],
                           PARTITIONER: Callable[[str], int]
                           ) -> List[Tuple[int, List[Tuple[int, str]]]]:
    """
    Распределенная группировка данных по ключу.
    """
    global reducers
    partitions = [dict() for _ in range(reducers)]  # Инициализация разделов
    for map_partition in map_partitions:  # Для каждого раздела данных после Map
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]  # Определяем раздел с помощью PARTITIONER
            p.setdefault(key, []).append(value)  # Добавляем данные в соответствующий раздел
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in enumerate(partitions)]

def PARTITIONER(obj: str) -> int:
    """
    Функция для определения разделителя (партиционера).
    """
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT,
                         MAP,
                         REDUCE,
                         PARTITIONER= PARTITIONER,
                         COMBINER = None
                         ):
    """
    Функция для выполнения операций MapReduce в распределенном формате.
    """
    # Применение Map к данным и выравнивание результатов
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    # Если указан COMBINER, применяем его к результатам Map после группировки
    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    # Выполнение распределенной группировки данных по ключу
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    # Применение Reduce к каждой группе данных и выравнивание результатов
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    # Подсчет количества отправленных по сети ключ-значений
    total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])

    print(f"{total_values_sent} key-value pairs were sent over a network.")

    return reduce_outputs

def INPUTFORMAT() -> Iterator:
    """
    Форматирование входных данных для MapReduce.
    """
    global maps

    def RECORDREADER(split: List[str]) -> Iterator[Tuple[str, str]]:
        """
        Генерирует ключ-значение для каждой строки документа.
        """
        for docid, document in enumerate(split):
            for lineid, line in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid, lineid), line)

    # Разделяем документы на части для Map
    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: str, line: str) -> Iterator[Tuple[str, str]]:
    """
    Функция Map для преобразования данных.
    """
    for word in line.split(" "):
        yield (word, word)

def REDUCE(key: str, value: Iterator[str]) -> Iterator[str]:
    """
    Функция Reduce для агрегации данных.
    """
    yield key

# Задание корпуса текстов
d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a dog"""
documents = [d1, d2, d3, d1, d2, d3]

# Данные делятся на :maps частей и потом обрабатываются :reducers редьюсерами
maps = 3
reducers = 2

# Выполнение MapReduce конвейера
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]
print(partitioned_output)

56 key-value pairs were sent over a network.
[(0, ['', 'what']), (1, ['a', 'dog', 'is', 'it'])]


#Операторы реляционной алгебры




### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.

In [ ]:
from collections import defaultdict
import random
from typing import List, Tuple, Callable


def MAP(el_list, predicate):
    """
    Реализует операцию MAP с настраиваемым предикатом для фильтрации кортежей.
    """
    mapped_result = defaultdict(list)
    for t in el_list:
        if predicate(t):  # Используем переданный предикат
            mapped_result[t].append(t)
    return list(mapped_result.items())

def REDUCE(mapped_items):
    """
    Выполняет операцию REDUCE, которая возвращает все элементы, сгруппированные по ключу.
    """
    reduced_result = []
    print(mapped_items)  # Выводим промежуточный результат
    for values in mapped_items:
        for value in values:
            reduced_result.append(value)  # Функция идентичности: возвращаем то же значение, что получили на вход
    return reduced_result

def C(t) -> bool:
    """
    Предикат для фильтрации кортежей с четным первым элементом.
    """
    return t[0] % 2 == 0

def RECORDREADER(count):
    """
    Генерация случайных записей (кортежей формата (x, y, z)).
    """
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

# Создание набора записей
record = RECORDREADER(5)
print(record)

# Разбиение записей на равные части
part_count = 5
record_partitioned = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitioned)

# Применение операции MAP и последующей REDUCE к разделенным записям
print(REDUCE(list(map(lambda x: MAP(x, C), record_partitioned))))

[(88, 22, 37), (66, 55, 37), (24, 77, 64), (42, 68, 57), (83, 52, 99)]
[[(88, 22, 37), (66, 55, 37), (24, 77, 64), (42, 68, 57), (83, 52, 99)]]
[[((88, 22, 37), [(88, 22, 37)]), ((66, 55, 37), [(66, 55, 37)]), ((24, 77, 64), [(24, 77, 64)]), ((42, 68, 57), [(42, 68, 57)])]]
[((88, 22, 37), [(88, 22, 37)]), ((66, 55, 37), [(66, 55, 37)]), ((24, 77, 64), [(24, 77, 64)]), ((42, 68, 57), [(42, 68, 57)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [ ]:
import random
from typing import Iterator, NamedTuple

# Создаем множество S, содержащее выбранные числа для фильтрации
S = set([25, 90, 69])  # Множество S используется для проверки наличия элементов

def MAP(t):
    """
    Функция MAP принимает кортеж чисел и фильтрует его, оставляя только элементы, присутствующие в множестве S.
    Возвращает кортеж, состоящий из двух одинаковых копий отфильтрованного набора элементов.
    """
    res_list = []
    for el in t:  # Перебираем элементы входного кортежа t
        if el in S:  # Проверяем, содержится ли элемент el в множестве S
            res_list.append(el)
    res = tuple(res_list)  # Преобразуем список в кортеж для неизменяемости
    return (res, res)

def REDUCE(key, values: Iterator[NamedTuple]):
    """
    Функция REDUCE принимает ключ и итерируемый объект значений, и проводит операцию редукции.
    В данной реализации возвращается кортеж, где и ключ, и значение равны переданному ключу.
    """
    return (key, key)

def RECORDREADER(count):
    """
    Функция RECORDREADER генерирует список записей, каждая из которых представляет собой кортеж из трех случайных чисел.
    Используется для создания тестовых данных, где параметр count определяет количество генерируемых записей.
    """
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
    """
    Функция group_by_key группирует элементы входного итерируемого объекта по ключу.
    Каждый элемент итерируемого объекта должен быть кортежем (ключ, значение), и функция возвращает пары, где каждому ключу соответствует список значений.
    """
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

# Применяем функцию MAP ко всем элементам, сгенерированным функцией RECORDREADER(100)
map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))
print("MAP output:", map_output)

# Группируем значения по ключу, используя функцию group_by_key
shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

# Применяем функцию REDUCE к результату группировки, извлекая из возвращаемого кортежа первый элемент
reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print("Reduce output:", reduce_output)

MAP output: [((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ())]
Shuffle output: [((), [(), (), (), (), (), (), (), (), (), ()])]
Reduce output: [()]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [ ]:
import random
from typing import Iterator, NamedTuple

def MAP(t):
    """
    Функция MAP принимает элемент t и возвращает кортеж, содержащий этот элемент дважды.
    Используется для подготовки данных к дальнейшей обработке.
    """
    return (t, t)  # Возвращаем кортеж из входного элемента и его же копии

def REDUCE(key, values: Iterator[NamedTuple]):
    """
    Функция REDUCE принимает ключ и список значений, связанных с этим ключом.
    В данной реализации просто возвращает кортеж, содержащий ключ дважды.
    """
    return (key, key)  # Возвращает кортеж с ключом и значением, равными переданному ключу

def RECORDREADER(count):
    """
    Функция RECORDREADER генерирует список случайных записей.
    Каждая запись представляет собой кортеж из трех случайных чисел в диапазоне от 0 до 100.
    """
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable):
    """
    Функция group_by_key группирует элементы входного итерируемого объекта по ключу.
    Ожидает, что элементы представлены в виде кортежей (ключ, значение).
    Возвращает пары (ключ, список значений).
    """
    t = {}  # Инициализируем словарь для группировки данных
    for (k2, v2) in iterable:  # Перебираем пары (ключ, значение)
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение в список, соответствующий ключу
    return t.items()  # Возвращаем сгруппированные данные в виде пар (ключ, список значений)


# Применяем функцию MAP к 10 сгенерированным записям
map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))
print("MAP output:", map_output)

# Группируем данные по ключу
shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

# Применяем REDUCE к сгруппированным данным
reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print("Reduce output:", reduce_output)

MAP output: [((88, 64, 8), (88, 64, 8)), ((80, 55, 29), (80, 55, 29)), ((19, 50, 98), (19, 50, 98)), ((64, 87, 63), (64, 87, 63)), ((26, 39, 2), (26, 39, 2)), ((32, 87, 70), (32, 87, 70)), ((47, 99, 55), (47, 99, 55)), ((70, 86, 88), (70, 86, 88)), ((12, 54, 1), (12, 54, 1)), ((17, 64, 90), (17, 64, 90))]
Shuffle output: [((88, 64, 8), [(88, 64, 8)]), ((80, 55, 29), [(80, 55, 29)]), ((19, 50, 98), [(19, 50, 98)]), ((64, 87, 63), [(64, 87, 63)]), ((26, 39, 2), [(26, 39, 2)]), ((32, 87, 70), [(32, 87, 70)]), ((47, 99, 55), [(47, 99, 55)]), ((70, 86, 88), [(70, 86, 88)]), ((12, 54, 1), [(12, 54, 1)]), ((17, 64, 90), [(17, 64, 90)])]
Reduce output: [(88, 64, 8), (80, 55, 29), (19, 50, 98), (64, 87, 63), (26, 39, 2), (32, 87, 70), (47, 99, 55), (70, 86, 88), (12, 54, 1), (17, 64, 90)]


### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [ ]:
import random
from typing import Iterator, NamedTuple

def MAP(t):
    """
    Функция MAP принимает элемент t и возвращает кортеж, содержащий этот элемент дважды.
    Используется для подготовки данных к дальнейшей обработке.
    """
    return (t, t)  # Возвращаем кортеж из входного элемента и его же копии

def REDUCE(key, values: Iterator[NamedTuple]):
    """
    Функция REDUCE принимает ключ и список значений, связанных с этим ключом.
    Если количество значений равно 2, возвращает кортеж, содержащий ключ дважды.
    """
    if len(values) == 2:
        return (key, key)  # Возвращаем кортеж с ключом и значением, равными переданному ключу

def RECORDREADER(count):
    """
    Функция RECORDREADER генерирует список случайных записей.
    Каждая запись представляет собой кортеж из двух случайных чисел в диапазоне от 0 до 3.
    """
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def group_by_key(iterable):
    """
    Функция group_by_key группирует элементы входного итерируемого объекта по ключу.
    Ожидает, что элементы представлены в виде кортежей (ключ, значение).
    Возвращает пары (ключ, список значений).
    """
    t = {}  # Инициализируем словарь для группировки данных
    for (k2, v2) in iterable:  # Перебираем пары (ключ, значение)
        t[k2] = t.get(k2, []) + [v2]  # Добавляем значение в список, соответствующий ключу
    return t.items()  # Возвращаем сгруппированные данные в виде пар (ключ, список значений)

# Применяем функцию MAP к 100 сгенерированным записям
map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))
print("MAP output:", map_output)

# Группируем данные по ключу
shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

# Применяем REDUCE к сгруппированным данным и фильтруем None
reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print("Reduce output:", reduce_output)

MAP output: [((1, 3), (1, 3)), ((2, 0), (2, 0)), ((1, 0), (1, 0)), ((3, 3), (3, 3)), ((1, 1), (1, 1)), ((2, 2), (2, 2)), ((2, 2), (2, 2)), ((2, 1), (2, 1)), ((1, 0), (1, 0)), ((2, 2), (2, 2))]
Shuffle output: [((1, 3), [(1, 3)]), ((2, 0), [(2, 0)]), ((1, 0), [(1, 0), (1, 0)]), ((3, 3), [(3, 3)]), ((1, 1), [(1, 1)]), ((2, 2), [(2, 2), (2, 2), (2, 2)]), ((2, 1), [(2, 1)])]
Reduce output: [(1, 0)]


### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [ ]:
import random
from typing import Iterator, NamedTuple

rels = [1, 2]  # Массив возможных идентификаторов представления

class Tuple:
    """
    Класс Tuple представляет объект с данными и идентификатором представления.
    """
    def __init__(self, data: tuple, rel_id: int):
        self.data = data  # Хранит кортеж данных
        self.rel_id = rel_id  # Хранит id представления

def get_random_tuple(count):
    """
    Генерирует объект Tuple с случайными данными.
    """
    data = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(count))
    rel_id = random.choice(rels)  # Выбираем случайный id представления
    return Tuple(data, rel_id)

def RECORDREADER(count):
    """
    Генерирует список случайных объектов Tuple.
    """
    return [get_random_tuple(3) for _ in range(count)]

def MAP(t: Tuple):
    """
    Функция MAP принимает объект Tuple и возвращает кортеж из данных и идентификатора представления.
    """
    return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[int]):
    """
    Функция REDUCE принимает ключ и список значений.
    Если список содержит только первый элемент из rels, возвращает кортеж (ключ, ключ).
    """
    if list(values) == [rels[0]]:  # Проверяем, что значения соответствуют первому элементу из rels
        return (key, key)

def group_by_key(iterable):
    """
    Функция группирует элементы входного итерируемого объекта по ключу.
    """
    t = {}
    for (k2, v2) in iterable:
        t.setdefault(k2, []).append(v2)
    return t.items()

# Применяем MAP к данным
map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))
print("MAP output:", map_output)

# Группируем результаты по ключу
shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

# Применяем REDUCE к сгруппированным данным и фильтруем None
reduce_output = [el[0] for el in map(lambda x: REDUCE(*x), shuffle_output) if el is not None]
print("Reduce output:", reduce_output)

MAP output: [(((3, 3), (1, 2), (0, 0)), 2), (((0, 2), (2, 0), (1, 3)), 1), (((1, 3), (1, 0), (1, 2)), 2), (((2, 0), (2, 0), (2, 3)), 2), (((1, 0), (0, 0), (1, 1)), 2), (((1, 0), (3, 1), (2, 0)), 1), (((3, 3), (2, 3), (1, 3)), 2), (((0, 2), (0, 0), (2, 1)), 1), (((0, 1), (2, 3), (2, 3)), 1), (((2, 3), (0, 2), (3, 2)), 1)]
Shuffle output: [(((3, 3), (1, 2), (0, 0)), [2]), (((0, 2), (2, 0), (1, 3)), [1]), (((1, 3), (1, 0), (1, 2)), [2]), (((2, 0), (2, 0), (2, 3)), [2]), (((1, 0), (0, 0), (1, 1)), [2]), (((1, 0), (3, 1), (2, 0)), [1]), (((3, 3), (2, 3), (1, 3)), [2]), (((0, 2), (0, 0), (2, 1)), [1]), (((0, 1), (2, 3), (2, 3)), [1]), (((2, 3), (0, 2), (3, 2)), [1])]
Reduce output: [((0, 2), (2, 0), (1, 3)), ((1, 0), (3, 1), (2, 0)), ((0, 2), (0, 0), (2, 1)), ((0, 1), (2, 3), (2, 3)), ((2, 3), (0, 2), (3, 2))]


### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:
import random
from typing import Iterator, NamedTuple

rels = [1, 2]  # Инициализация списка идентификаторов связей

class Tuple:
    """
    Класс для хранения данных и идентификатора связи.
    """
    def __init__(self, data: tuple, rel_id: int):
        self.data = data  # Кортеж данных (x, y)
        self.rel_id = rel_id  # Идентификатор связи

def get_random_tuple():
    """
    Генерация случайного кортежа с данными и идентификатором связи.
    """
    data = (random.randint(0, 3), random.randint(0, 3))  # Генерируем случайный кортеж (x, y)
    rel_id = random.choice(rels)  # Случайный идентификатор связи
    return Tuple(data, rel_id)  # Возвращаем объект Tuple

def RECORDREADER(count):
    """
    Создание списка случайных кортежей заданного количества.
    """
    return [get_random_tuple() for _ in range(count)]  # Генерация списка объектов Tuple

def MAP(t: Tuple):
    """
    Преобразование данных кортежа в зависимости от идентификатора связи.
    """
    if t.rel_id == rels[0]:  # Если rel_id совпадает с первым значением в rels
        return (t.data[1], (t.rel_id, t.data[0]))  # Меняем порядок элементов
    else:
        return (t.data[0], (t.rel_id, t.data[1]))  # Меняем порядок по другому условию

def REDUCE(key, values: Iterator[NamedTuple]):
    """
    Объединение данных по ключу.
    """
    res = []
    for v in values:
        res.append((v[0], key, v[1]))  # Создаем новый список значений
    return res

def group_by_key(iterable):
    """
    Группировка данных по ключу.
    """
    t = {}
    for (k2, v2) in iterable:
        t.setdefault(k2, []).append(v2)  # Добавляем значения в список по ключу
    return t.items()

# Применение функции MAP к каждому элементу списка
map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))
print("MAP output:", map_output)

# Группировка данных по ключу
shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

# Применение функции REDUCE к каждой группе
reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print("Reduce output:", reduce_output)

MAP output: [(1, (1, 3)), (2, (2, 2)), (0, (1, 3)), (0, (1, 3)), (1, (1, 2)), (2, (1, 0)), (1, (1, 3)), (3, (2, 0)), (3, (1, 3)), (1, (1, 0))]
Shuffle output: [(1, [(1, 3), (1, 2), (1, 3), (1, 0)]), (2, [(2, 2), (1, 0)]), (0, [(1, 3), (1, 3)]), (3, [(2, 0), (1, 3)])]
Reduce output: [[(1, 1, 3), (1, 1, 2), (1, 1, 3), (1, 1, 0)], [(2, 2, 2), (1, 2, 0)], [(1, 0, 3), (1, 0, 3)], [(2, 3, 0), (1, 3, 3)]]


### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [ ]:
import random
from typing import Iterator, NamedTuple

rels = [1, 2]  # Инициализация списка идентификаторов связей

class Tuple:  # Определение класса кортежа, хранящего данные и идентификатор связи
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple():  # Функция для генерации случайного кортежа с тремя случайными значениями от 0 до 3
    return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):  # Функция для создания списка случайных кортежей заданного количества
    return [get_random_tuple() for i in range(count)]

def MAP(t: tuple):  # Функция для преобразования входного кортежа, возвращающая первые два элемента кортежа
    return (t[0], t[1])

def tetta(values):  # Функция для вычисления суммы значений в переданном списке
    return sum(values)

def REDUCE(key, values: Iterator[NamedTuple]):  # Функция для вычисления суммы значений в переданных значениях и возврата пары (ключ, сумма)
    x = tetta(values)
    return (key, x)

def group_by_key(iterable):  # Функция для группировки данных по ключу
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

map_output = list(map(lambda x: MAP(x), RECORDREADER(10)))  # Применение функции MAP к каждому элементу
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  # Группировка данных по ключу
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))  # Применение функции REDUCE к каждой группе
print("Reduce output:", reduce_output)

MAP output: [(3, 3), (0, 3), (0, 3), (0, 2), (2, 0), (3, 2), (2, 1), (0, 0), (2, 1), (0, 3)]
Shuffle output: [(3, [3, 2]), (0, [3, 3, 2, 0, 3]), (2, [0, 1, 1])]
Reduce output: [(3, 5), (0, 11), (2, 2)]


### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [ ]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [8]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [ ]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [ ]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [ ]:
import numpy as np  # Импорт библиотеки numpy под псевдонимом np

# Инициализация размеров матриц
I = 2  # Количество строк первой матрицы
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4*10  # Количество столбцов второй матрицы

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Случайная матрица размером IxJ
big_mat = np.random.rand(J, K)  # Случайная матрица размером JxK

# Вычисление эталонного решения матричного умножения
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
    """
    Генерация значений для элементов двух матриц в виде кортежей (метка, индекс строк и столбцов, значение).
    """
    for i in range(small_mat.shape[0]):  # Проход по строкам первой матрицы
        for j in range(small_mat.shape[1]):  # Проход по столбцам первой матрицы
            yield ((0, i, j), small_mat[i, j])  # Генерация кортежа для первой матрицы

    for j in range(big_mat.shape[0]):  # Проход по строкам второй матрицы
        for k in range(big_mat.shape[1]):  # Проход по столбцам второй матрицы
            yield ((1, j, k), big_mat[j, k])  # Генерация кортежа для второй матрицы

def MAP_JOIN(k1, v1):
    """
    Соединение элементов матриц по индексам для дальнейшей обработки.
    """
    (mat_num, i, j) = k1  # Распаковка ключа
    w = v1  # Присвоение значения
    if mat_num == 0:  # Если элемент из первой матрицы
        yield (j, (mat_num, i, w))  # Генерация ключа для первой матрицы
    else:  # Если элемент из второй матрицы
        yield (i, (mat_num, j, w))  # Генерация ключа для второй матрицы

def REDUCE_JOIN(key, values):
    """
    Сведение значений из двух матриц по ключам для умножения соответствующих элементов.
    """
    from_first_mat = [v for v in values if v[0] == 0]  # Извлечение значений из первой матрицы
    from_second_mat = [v for v in values if v[0] == 1]  # Извлечение значений из второй матрицы
    for f in from_first_mat:  # Перебор элементов из первой матрицы
        for s in from_second_mat:  # Перебор элементов из второй матрицы
            yield ((f[1], s[1]), f[2] * s[2])  # Генерация произведения соответствующих элементов

def MAP_MUL(k1, v1):
    """
    Умножение значений и генерация новых ключей для хранения результата.
    """
    (i, k) = k1  # Распаковка ключа
    yield ((i, k), v1)  # Генерация кортежа с результатом умножения

def REDUCE_MUL(key, values):
    """
    Суммирование значений для получения итогового результата по ключу.
    """
    res_el_value = 0  # Инициализация переменной для хранения результата
    for v in values:  # Перебор значений
        res_el_value += v  # Суммирование значений
    yield (key, res_el_value)  # Генерация кортежа с суммой значений

def GET_JOINED():
    """
    Генерация всех элементов, объединённых после MapReduce.
    """
    for j in joined:  # Проход по всем элементам после объединения
        yield j  # Генерация каждого элемента

# Применение алгоритма MapReduce к функциям
joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)  # Применение MapReduce для объединения матриц
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)  # Применение MapReduce для умножения и суммирования

# Проверка правильности решения
np.allclose(reference_solution, asmatrix(solution))  # Сравнение результата с эталонным решением

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [9]:
# Определение размеров матриц
I = 2
J = 3
K = 4

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Первая случайная матрица размером IxJ
big_mat = np.random.rand(J, K)  # Вторая случайная матрица размером JxK

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
    """
    Разглаживание вложенных итерируемых объектов в одномерный поток элементов.
    """
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    """
    Группировка элементов по ключу в словарь.
    """
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    """
    Распределенная группировка элементов по ключу с использованием указанного разделителя.
    """
    global reducers
    partitions = [dict() for _ in range(reducers)]  # Инициализация списков для каждой группы
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]  # Определение раздела для ключа
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    """
    Определение разделителя для ключа, который определяет, в какую группу попадет элемент.
    """
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    """
    Выполнение распределенного MapReduce для обработки данных.
    """
    # Разделение данных на части и применение MAP функции
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    # Применение COMBINER, если он определен
    if COMBINER != None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    # Распределенная группировка данных по ключу
    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    # Применение REDUCE функции
    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    # Вывод количества ключей, переданных по сети
    print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs

def asmatrix(reduce_output):
    """
    Преобразование данных после REDUCE в матрицу.
    """
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for ((i, k), vw) in reduce_output:
        mat[i, k] = vw
    return mat

def INPUTFORMAT():
    """
    Генератор для создания исходных данных (матриц).
    """
    first_mat = []
    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            first_mat.append(((0, i, j), small_mat[i, j]))  # Первая матрица
    yield first_mat

    second_mat = []
    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            second_mat.append(((1, j, k), big_mat[j, k]))  # Вторая матрица
    yield second_mat

def MAP_JOIN(k1, v1):
    """
    MAP функция для соединения элементов матриц по индексу.
    """
    (mat_num, i, j) = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))  # Для первой матрицы
    else:
        yield (i, (mat_num, j, w))  # Для второй матрицы

def REDUCE_JOIN(key, values):
    """
    REDUCE функция для умножения соответствующих значений из двух матриц.
    """
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
    """
    Генератор для получения всех соединенных значений после обработки.
    """
    for j in joined:
        print("aa", j)
        yield j[1]

def MAP_MUL(k1, v1):
    """
    MAP функция для умножения значений по ключу.
    """
    yield (k1, v1)

def REDUCE_MUL(key, values):
    """
    REDUCE функция для суммирования значений по ключу.
    """
    res_val = 0
    for v in values:
        res_val += v  # Суммирование значений
    yield (key, res_val)

maps = 4  # Количество Map задач
reducers = 2  # Количество Reducers

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

# Формирование итогового решения
solution = []
for p in pre_result:
    for v in p[1]:
        solution.append(v)

print(solution)

# Проверка результата на соответствие эталонному решению
np.allclose(reference_solution, asmatrix(solution))  # Сравнение с эталонным результатом

18 key-value pairs were sent over a network.
[(0, [((0, 0), 0.3652804124358081), ((0, 1), 0.18321077739056962), ((0, 2), 0.0913956789399809), ((0, 3), 0.06572566184135892), ((1, 0), 0.6815689261212347), ((1, 1), 0.3418490796351448), ((1, 2), 0.170533246860563), ((1, 3), 0.12263610977961939), ((0, 0), 0.2064393958152946), ((0, 1), 0.121660444152244), ((0, 2), 0.5699970607217889), ((0, 3), 0.04099983809959785), ((1, 0), 0.11660869084290779), ((1, 1), 0.0687207258281891), ((1, 2), 0.32196670006989464), ((1, 3), 0.02315903622311933)]), (1, [((0, 0), 0.09532239350997322), ((0, 1), 0.09884218024971253), ((0, 2), 0.5589043509031115), ((0, 3), 0.4802445412996089), ((1, 0), 0.05433816114969145), ((1, 1), 0.0563446019453319), ((1, 2), 0.3186012600854331), ((1, 3), 0.2737615403422231)])]
aa (0, [((0, 0), 0.3652804124358081), ((0, 1), 0.18321077739056962), ((0, 2), 0.0913956789399809), ((0, 3), 0.06572566184135892), ((1, 0), 0.6815689261212347), ((1, 1), 0.3418490796351448), ((1, 2), 0.17053324686

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [11]:
# Определение размеров матриц
I = 2  # Количество строк первой матрицы и строк в итоговой матрице
J = 3  # Количество столбцов первой матрицы и строк второй матрицы
K = 4  # Количество столбцов второй матрицы и столбцов в итоговой матрице

# Генерация случайных матриц
small_mat = np.random.rand(I, J)  # Генерация первой матрицы
big_mat = np.random.rand(J, K)  # Генерация второй матрицы

# Получение эталонного решения через умножение матриц
reference_solution = np.matmul(small_mat, big_mat)

# Функция для "разглаживания" вложенных итерируемых объектов
def flatten(nested_iterable):
    """
    Разглаживает вложенные итерируемые объекты в одномерный список.
    """
    for iterable in nested_iterable:
        for element in iterable:
            yield element

# Функция для группировки элементов по ключу
def groupbykey(iterable):
    """
    Группирует элементы по ключу, создавая словарь с ключами и соответствующими им значениями.
    """
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

# Функция для распределенной группировки элементов по ключу
def groupbykey_distributed(map_partitions, PARTITIONER):
    """
    Распределяет элементы по нескольким партициям с использованием функции PARTITIONER.
    """
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for (k2, v2) in map_partition:
            p = partitions[PARTITIONER(k2)]
            p[k2] = p.get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

# Функция для определения разделителя (по ключу)
def PARTITIONER(obj):
    """
    Определяет партицию для объекта на основе хэш-функции.
    """
    global reducers
    return hash(obj) % reducers

# Функция для выполнения MapReduce на распределенных данных
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    """
    Выполняет MapReduce на распределенных данных с использованием функций MAP и REDUCE.
    """
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)  # shuffle

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
    return reduce_outputs

# Функция для преобразования результата REDUCE в матрицу
def asmatrix(reduce_output):
    """
    Преобразует результат REDUCE в матрицу, формируя ее из кортежей (i, k) и значений.
    """
    reduce_output = list(reduce_output)
    I = max(i for ((i,k), vw) in reduce_output) + 1
    K = max(k for ((i,k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I,K))
    for ((i,k), vw) in reduce_output:
        mat[i,k] = vw
    return mat

# Генератор для вводных данных
def INPUTFORMAT():
    """
    Генерирует вводные данные для MapReduce, включая первую и вторую матрицы.
    """
    first_mat = []

    for i in range(small_mat.shape[0]):
        for j in range(small_mat.shape[1]):
            first_mat.append(((0, i, j), small_mat[i,j]))  # первая матрица

    global maps
    split_size = int(np.ceil(len(first_mat) / maps))

    for i in range(0, len(first_mat), split_size):
        yield first_mat[i:i+split_size]

    second_mat = []

    for j in range(big_mat.shape[0]):
        for k in range(big_mat.shape[1]):
            second_mat.append(((1, j, k), big_mat[j,k]))  # вторая матрица

    split_size = int(np.ceil(len(second_mat) / maps))

    for i in range(0, len(second_mat), split_size):
        yield second_mat[i:i+split_size]

# MAP функция для соединения матриц
def MAP_JOIN(k1, v1):
    """
    Преобразует элементы для соединения матриц на основе их индексов.
    """
    (mat_num, i, j) = k1
    w = v1

    if mat_num == 0:
        yield (j, (mat_num, i, w))
    else:
        yield (i, (mat_num, j, w))

# REDUCE функция для соединения матриц
def REDUCE_JOIN(key, values):
    """
    Выполняет соединение элементов из обеих матриц по соответствующим индексам.
    """
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]

    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])

# Генератор для получения соединенных данных
def GET_JOINED():
    """
    Генерирует соединенные данные для дальнейших вычислений.
    """
    for j in joined:
        print("aa", j)
        yield j[1]

# MAP функция для умножения значений
def MAP_MUL(k1, v1):
    """
    Процесс умножения значений в MapReduce.
    """
    yield (k1, v1)

# REDUCE функция для умножения значений
def REDUCE_MUL(key, values):
    """
    Выполняет суммирование значений для получения результата умножения матриц.
    """
    res_val = 0
    for v in values:
        res_val += v
    yield (key, res_val)

maps = 3  # Количество карт (партиций)
reducers = 2  # Количество редьюсеров

# Выполнение MapReduce для соединения матриц
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

# Выполнение MapReduce для умножения значений
mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:
    for v in p[1]:
        solution.append(v)

print(solution)

# Проверка результата на соответствие эталонному решению
np.allclose(reference_solution, asmatrix(solution))

18 key-value pairs were sent over a network.
[(0, [((0, 0), 0.0916670624616288), ((0, 1), 0.07612792714057096), ((0, 2), 0.00421767328094783), ((0, 3), 0.09510353969150907), ((1, 0), 0.5995560253783815), ((1, 1), 0.4979210219134225), ((1, 2), 0.02758604193003554), ((1, 3), 0.6220325897399022), ((0, 0), 0.07465707863767518), ((0, 1), 0.8807123916418103), ((0, 2), 0.5397691592620495), ((0, 3), 0.8365389216085857), ((1, 0), 0.06960007713083256), ((1, 1), 0.8210561075640131), ((1, 2), 0.5032071412786956), ((1, 3), 0.7798747892275435)]), (1, [((0, 0), 0.2793120251582633), ((0, 1), 0.5094013297321783), ((0, 2), 0.6821290097013052), ((0, 3), 0.042362937349492404), ((1, 0), 0.2234842560492663), ((1, 1), 0.4075842317966699), ((1, 2), 0.5457877947658744), ((1, 3), 0.03389560307061094)])]
aa (0, [((0, 0), 0.0916670624616288), ((0, 1), 0.07612792714057096), ((0, 2), 0.00421767328094783), ((0, 3), 0.09510353969150907), ((1, 0), 0.5995560253783815), ((1, 1), 0.4979210219134225), ((1, 2), 0.027586041

True